# 🚗 Nhận Dạng Biển Số Xe Việt Nam

## Vietnamese License Plate Recognition (Fixed for PyTorch 2.6)

### 🎯 Tính năng:
- Phát hiện biển số xe trong ảnh
- Nhận dạng ký tự và số trên biển số
- Hỗ trợ biển số 1 dòng và 2 dòng

### 📋 Hướng dẫn:
1. **Chọn GPU Runtime**: Runtime → Change runtime type → GPU
2. **Chạy từng cell theo thứ tự** từ trên xuống dưới
3. **Đợi cell hoàn thành** trước khi chạy cell tiếp theo

---

## 1. Setup và Clone Repository

In [ ]:
import os
import sys
import torch

# Fix PyTorch 2.6 weights_only issue
print(f"🔧 PyTorch version: {torch.__version__}")
if hasattr(torch.serialization, 'add_safe_globals'):
    print("🔧 Fixing PyTorch 2.6 weights_only issue...")
    # Add safe globals for YOLOv5
    try:
        from models.yolo import Model
        torch.serialization.add_safe_globals([Model])
        print("✅ PyTorch fix applied!")
    except:
        print("⚠️ Will apply fix after cloning repository")

# Clone repository
if not os.path.exists('BsxGoogleColab'):
    print("🔄 Cloning repository...")
    !git clone https://github.com/ngoclam99/BsxGoogleColab.git
    print("✅ Clone successful!")
else:
    print("✅ Repository already exists!")

# Change directory
os.chdir('BsxGoogleColab')
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
if os.path.join(os.getcwd(), 'yolov5') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'yolov5'))

## 2. Install Dependencies và Fix PyTorch

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install -q opencv-python-headless matplotlib numpy Pillow torch torchvision tqdm pyyaml requests pandas seaborn

# Apply PyTorch 2.6 fix
print("🔧 Applying PyTorch 2.6 compatibility fix...")
try:
    import torch
    # Import YOLOv5 models to add to safe globals
    sys.path.append('yolov5')
    from models.yolo import Model
    from models.common import *
    from utils.general import *
    
    # Add safe globals for PyTorch 2.6
    if hasattr(torch.serialization, 'add_safe_globals'):
        torch.serialization.add_safe_globals([
            Model, Conv, Bottleneck, SPP, DWConv, Focus, 
            BottleneckCSP, C3, nn.Upsample, Concat, Detect
        ])
        print("✅ PyTorch 2.6 fix applied successfully!")
    else:
        print("ℹ️ PyTorch version doesn't need fix")
        
except Exception as e:
    print(f"⚠️ Fix attempt failed: {e}")
    print("💡 Will try alternative loading method")

print("✅ Installation complete!")

## 3. Import Libraries và Check Files

In [ ]:
# Import libraries
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Import helper functions
try:
    import function.utils_rotate as utils_rotate
    import function.helper as helper
    print("✅ Helper functions imported successfully!")
except ImportError as e:
    print(f"❌ Error importing helpers: {e}")

# Check files
required_files = {
    'LP Detector Model': 'model/LP_detector.pt',
    'LP OCR Model': 'model/LP_ocr.pt', 
    'YOLOv5 Directory': 'yolov5',
    'Test Images': 'test_image'
}

print("🔍 Checking required files...")
all_files_ok = True
for name, path in required_files.items():
    if os.path.exists(path):
        if os.path.isfile(path):
            size = os.path.getsize(path) / (1024*1024)  # MB
            print(f"✅ {name}: {path} ({size:.1f} MB)")
        else:
            count = len(os.listdir(path)) if os.path.isdir(path) else 0
            print(f"✅ {name}: {path} ({count} items)")
    else:
        print(f"❌ {name}: {path} - MISSING")
        all_files_ok = False

if all_files_ok:
    print("\n🟢 All required files are ready!")
else:
    print("\n🔴 Some files are missing! Check your repository.")

print("✅ Setup complete!")

## 4. Load Models (với Fix cho PyTorch 2.6)

In [ ]:
# Load YOLOv5 models with PyTorch 2.6 fix
print("🤖 Loading models with PyTorch 2.6 compatibility...")
print("⏳ This may take 2-3 minutes...")

def load_model_safe(model_path, model_name):
    """Load model with PyTorch 2.6 compatibility"""
    try:
        # Method 1: Try with weights_only=False (for trusted models)
        print(f"📥 Loading {model_name} (Method 1)...")
        
        # Temporarily set weights_only to False for trusted models
        import torch
        original_load = torch.load
        
        def safe_load(*args, **kwargs):
            kwargs['weights_only'] = False
            return original_load(*args, **kwargs)
        
        torch.load = safe_load
        
        model = torch.hub.load('yolov5', 'custom', 
                              path=model_path, 
                              force_reload=True, 
                              source='local',
                              trust_repo=True)
        
        # Restore original torch.load
        torch.load = original_load
        
        print(f"✅ {model_name} loaded successfully!")
        return model
        
    except Exception as e:
        print(f"❌ Method 1 failed for {model_name}: {e}")
        
        try:
            # Method 2: Direct model loading
            print(f"📥 Loading {model_name} (Method 2)...")
            
            import sys
            sys.path.append('yolov5')
            from models.experimental import attempt_load
            
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = attempt_load(model_path, map_location=device)
            model.eval()
            
            print(f"✅ {model_name} loaded with Method 2!")
            return model
            
        except Exception as e2:
            print(f"❌ Method 2 also failed for {model_name}: {e2}")
            return None

# Load models
yolo_LP_detect = load_model_safe('model/LP_detector.pt', 'License Plate Detector')
yolo_license_plate = load_model_safe('model/LP_ocr.pt', 'License Plate OCR')

# Set confidence if models loaded successfully
if yolo_license_plate is not None:
    try:
        yolo_license_plate.conf = 0.60
        print("🔧 Confidence threshold set to 0.60")
    except:
        print("⚠️ Could not set confidence threshold")

# Check final status
if yolo_LP_detect is not None and yolo_license_plate is not None:
    print("\n🎉 ALL MODELS LOADED SUCCESSFULLY!")
    print("🟢 Ready for license plate recognition!")
else:
    print("\n❌ FAILED TO LOAD MODELS!")
    print("💡 Solutions:")
    print("   1. Runtime → Restart Runtime and try again")
    print("   2. Make sure you have GPU runtime selected")
    print("   3. Check if model files exist in repository")

## 5. Detection Function

In [ ]:
def detect_license_plate(image_path):
    """Detect and recognize license plate with error handling"""
    
    # Check if models are loaded
    if yolo_LP_detect is None or yolo_license_plate is None:
        print("❌ Models not loaded! Please run the model loading cell first.")
        return None, None
    
    print(f"🔍 Processing: {image_path}")
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None
    
    try:
        # Detect license plates
        print("📸 Detecting license plates...")
        plates = yolo_LP_detect(img, size=640)
        
        # Handle different model types
        if hasattr(plates, 'pandas'):
            list_plates = plates.pandas().xyxy[0].values.tolist()
        else:
            # For direct model loading
            list_plates = []
            if len(plates) > 0:
                for det in plates:
                    if len(det):
                        for *xyxy, conf, cls in det:
                            if conf > 0.5:  # confidence threshold
                                list_plates.append([float(x) for x in xyxy] + [float(conf), int(cls)])
        
        list_read_plates = set()
        
        print(f"🎯 Found {len(list_plates)} license plate regions")
        
        if len(list_plates) == 0:
            # Try reading entire image
            print("🔄 Trying to read entire image...")
            lp = helper.read_plate(yolo_license_plate, img)
            if lp != "unknown":
                list_read_plates.add(lp)
        else:
            # Process each detected plate
            for i, plate in enumerate(list_plates):
                print(f"📝 Reading plate {i+1}/{len(list_plates)}...")
                
                x = int(plate[0])
                y = int(plate[1])
                w = int(plate[2] - plate[0])
                h = int(plate[3] - plate[1])
                
                # Crop license plate
                crop_img = img[y:y+h, x:x+w]
                
                # Draw rectangle
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                # Try different image processing
                found = False
                for cc in range(0, 2):
                    for ct in range(0, 2):
                        try:
                            processed_img = utils_rotate.deskew(crop_img, cc, ct)
                            lp = helper.read_plate(yolo_license_plate, processed_img)
                            if lp != "unknown":
                                list_read_plates.add(lp)
                                found = True
                                break
                        except Exception as e:
                            print(f"⚠️ Error processing variation {cc},{ct}: {e}")
                            continue
                    if found:
                        break
        
        # Add text to image
        if list_read_plates:
            y_offset = 30
            for plate_text in list_read_plates:
                cv2.putText(img, str(plate_text), (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                y_offset += 40
        
        if list_read_plates:
            print(f"✅ Result: {list_read_plates}")
        else:
            print("❌ No license plate recognized")
        
        return img, list_read_plates
        
    except Exception as e:
        print(f"❌ Error during detection: {e}")
        import traceback
        traceback.print_exc()
        return None, None

print("✅ Detection function ready!")

## 6. Test với Ảnh Mẫu

In [ ]:
# Import glob if not already imported
try:
    glob
except NameError:
    import glob
    print("📦 Imported glob module")

# Find test images
test_images = glob.glob("test_image/*.jpg") + glob.glob("test_image/*.png") + glob.glob("test_image/*.jpeg")

if test_images:
    print(f"🖼️ Found {len(test_images)} test images:")
    for i, img in enumerate(test_images):
        print(f"   {i}: {img}")
    
    # Test first image
    test_index = 0  # Change this to test different images
    
    if test_index < len(test_images):
        test_img = test_images[test_index]
        
        print(f"\n🚀 Testing: {test_img}")
        print("=" * 50)
        
        result_img, detected_plates = detect_license_plate(test_img)
        
        if result_img is not None:
            print("\n" + "=" * 50)
            print(f"🎯 FINAL RESULT: {detected_plates if detected_plates else 'NO DETECTION'}")
            print("=" * 50)
            
            # Display results
            fig, axes = plt.subplots(1, 2, figsize=(16, 8))
            
            # Original image
            original = cv2.imread(test_img)
            axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            axes[0].set_title("🖼️ Original Image", fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            # Result image
            axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            result_title = f"🎯 Result: {detected_plates if detected_plates else 'No detection'}"
            axes[1].set_title(result_title, fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Summary
            print(f"\n📊 SUMMARY:")
            print(f"   📁 File: {test_img}")
            print(f"   📏 Size: {original.shape[1]}x{original.shape[0]}")
            print(f"   🔢 Plates found: {len(detected_plates) if detected_plates else 0}")
            if detected_plates:
                for i, plate in enumerate(detected_plates, 1):
                    print(f"   🚗 Plate {i}: {plate}")
        else:
            print("❌ Failed to process image")
    else:
        print(f"❌ Invalid index {test_index}. Choose from 0 to {len(test_images)-1}")
else:
    print("❌ No test images found in test_image/ directory")
    print("💡 Make sure your repository has test images")

## 7. Upload Ảnh Của Bạn

In [ ]:
from google.colab import files

def upload_and_test():
    """Upload and test your own image"""
    print("📤 Select image file from your computer...")
    print("💡 Supported formats: .jpg, .jpeg, .png")
    
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Processing uploaded image: {filename}")
        print("=" * 50)
        
        # Save uploaded file
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])
        
        # Process image
        result_img, detected_plates = detect_license_plate(filename)
        
        if result_img is not None:
            print("\n" + "=" * 50)
            print(f"🎯 RESULT: {detected_plates if detected_plates else 'NO DETECTION'}")
            print("=" * 50)
            
            # Display results
            fig, axes = plt.subplots(1, 2, figsize=(16, 8))
            
            # Original
            original = cv2.imread(filename)
            axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            axes[0].set_title("🖼️ Your Image", fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            # Result
            axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            result_title = f"🎯 Result: {detected_plates if detected_plates else 'No detection'}"
            axes[1].set_title(result_title, fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Summary
            print(f"\n📊 SUMMARY:")
            print(f"   📁 File: {filename}")
            print(f"   📏 Size: {original.shape[1]}x{original.shape[0]}")
            if detected_plates:
                for i, plate in enumerate(detected_plates, 1):
                    print(f"   🚗 License Plate {i}: {plate}")
            else:
                print("   ❌ No license plate detected")
                print("   💡 Try with a clearer image or different angle")
        else:
            print("❌ Failed to process uploaded image")

# Uncomment the line below to upload and test your image
# upload_and_test()

## 📝 Hướng Dẫn Sử Dụng

### 🚀 Cách sử dụng:
1. **Chạy cells 1-4** để setup và load models
2. **Chạy cell 5** để định nghĩa hàm detection
3. **Chạy cell 6** để test với ảnh mẫu
4. **Chạy cell 7** để upload ảnh của bạn (uncomment dòng cuối)

### 💡 Tips để có kết quả tốt:
- **Chọn GPU Runtime** để xử lý nhanh hơn
- **Sử dụng ảnh rõ nét** và chất lượng cao
- **Biển số không bị che khuất** hoặc mờ
- **Góc chụp thẳng** cho kết quả tốt nhất
- **Ánh sáng đủ** để nhìn rõ ký tự

### 🔧 Khắc phục sự cố:
- **Models không load được**: 
  - Runtime → Restart Runtime và chạy lại
  - Đảm bảo đã chọn GPU runtime
- **Lỗi import**: Kiểm tra cấu trúc repository
- **Không nhận dạng được**: 
  - Thử ảnh khác rõ nét hơn
  - Kiểm tra biển số có bị che không
  - Thử góc chụp khác

### 🆘 Nếu vẫn gặp lỗi PyTorch 2.6:
1. **Restart Runtime** và chạy lại từ đầu
2. **Đảm bảo GPU runtime** được chọn
3. **Kiểm tra repository** có đầy đủ files không

---
**🚗 Chúc bạn nhận dạng biển số thành công! 🎉**